In [1]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [2]:
from aif360.algorithms.postprocessing.reject_option_classification import RejectOptionClassification
from aif360.datasets import BinaryLabelDataset, AdultDataset

pip install 'aif360[AdversarialDebiasing]'
pip install 'aif360[AdversarialDebiasing]'
pip install 'aif360[Reductions]'
pip install 'aif360[Reductions]'
pip install 'aif360[inFairness]'
pip install 'aif360[Reductions]'


In [3]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

# TODO: change the import method
import sys
import os
repo_root = os.path.dirname(os.getcwd())
sys.path.insert(0, repo_root)
repair_folder = os.path.join(repo_root, "humancompatible", "repair")
sys.path.insert(0, repair_folder)
from humancompatible.repair.cost import *
from humancompatible.repair.coupling_utils import *
from humancompatible.repair.data_analysis import *
from humancompatible.repair.group_blind_repair import *
from humancompatible.repair.metrics import *

In [4]:
import os
path=os.path.dirname(os.getcwd())

In [5]:
class ROCpostprocess:
    def __init__(self,X_val,y_val,var_list,prediction_model,favorable_label):
        self.X_val =X_val
        self.y_val =y_val
        self.model = prediction_model
        self.positive_index = 1 # positive label
        self.var_list = var_list
        self.var_dim=len(self.var_list)
        self.ROC = self.buildROCusingval()
        self.favorable_label = favorable_label

    def buildbinarydata(self,X,y):
        df=pd.DataFrame(np.concatenate((X,y.reshape(-1,1)), axis=1),columns=self.var_list+['S','W','Y'])
        binaryLabelDataset = BinaryLabelDataset(
                            # favorable_label=self.favorable_label,
                            # unfavorable_label=0,
                            df=df[self.var_list+['S','W','Y']], #df_test.drop('X',axis=1), #[x_list+['S','W','Y']],
                            label_names=['Y'],
                            instance_weights_name=['W'],
                            protected_attribute_names=['S'],
                            privileged_protected_attributes=[np.array([1.0])],
                            unprivileged_protected_attributes=[np.array([0.])])
        return binaryLabelDataset,df

    def buildROCusingval(self):
        dataset_val = self.buildbinarydata(self.X_val,self.y_val)[0]
        dataset_val_pred = dataset_val.copy(deepcopy=True)
        dataset_val_pred.scores = self.model.predict_proba(dataset_val.features[:,0:self.var_dim])[:,self.positive_index].reshape(-1,1)
        privileged_groups = [{'S': 1}]
        unprivileged_groups = [{'S': 0}]
        # Metric used (should be one of allowed_metrics)
        metric_name = "Statistical parity difference"
        # Upper and lower bound on the fairness metric used
        metric_ub = 0.05
        metric_lb = -0.05
        ROC = RejectOptionClassification(unprivileged_groups=unprivileged_groups, 
                                        privileged_groups=privileged_groups, 
                                        low_class_thresh=0.01, high_class_thresh=0.99,
                                        num_class_thresh=50, num_ROC_margin=10,
                                        metric_name=metric_name,
                                        metric_ub=metric_ub, metric_lb=metric_lb)
        ROC = ROC.fit(dataset_val, dataset_val_pred)
        print("Optimal classification threshold (with fairness constraints) = %.4f" % ROC.classification_threshold)
        print("Optimal ROC margin = %.4f" % ROC.ROC_margin)
        return ROC

    def postprocess(self,X_test,y_test,tv_origin): # the tv distance won't change
        dataset_test_pred,df_test = self.buildbinarydata(X_test,y_test) #.copy(deepcopy=True)
        dataset_test_pred.scores = self.model.predict_proba(X_test[:,0:self.var_dim])[:,self.positive_index].reshape(-1,1)
        dataset_test_pred_transf = self.ROC.predict(dataset_test_pred)
        y_pred = dataset_test_pred_transf.labels
        # return dataset_test_pred_transf.convert_to_dataframe()[0]

        di = DisparateImpact_postprocess(df_test,y_pred,favorable_label=self.favorable_label)
        f1_macro = f1_score(df_test['Y'], y_pred, average='macro',sample_weight=df_test['W'])
        f1_micro = f1_score(df_test['Y'], y_pred, average='micro',sample_weight=df_test['W'])
        f1_weighted = f1_score(df_test['Y'], y_pred, average='weighted',sample_weight=df_test['W'])
        new_row=pd.Series({'DI':di,'f1 macro':f1_macro,'f1 micro':f1_micro,'f1 weighted':f1_weighted,
                           'TV distance':tv_origin,'method':'ROC'})
        return new_row.to_frame().T

In [6]:
from humancompatible.repair.proj_postprocess import Projpostprocess

In [7]:
def load_data(data_path,var_list,pa):
    column_names = ['age', 'workclass', 'fnlwgt', 'education',
                'education-num', 'marital-status', 'occupation', 'relationship',
                'race', 'sex', 'capital-gain', 'capital-loss', 'hours-per-week',
                'native-country', 'Y']
    na_values=['?']
    pa_dict={'Male':1,'Female':0,'White':1,'Black':0}
    label_dict={'>50K.':1,'>50K':1,'<=50K.':0,'<=50K':0}
    train_path = os.path.join(data_path, 'adult.data')
    test_path = os.path.join(data_path, 'adult.test')
    train = pd.read_csv(train_path, header=None,names=column_names,
                    skipinitialspace=True, na_values=na_values)
    test = pd.read_csv(test_path, header=0,names=column_names,
                    skipinitialspace=True, na_values=na_values)
    messydata = pd.concat([test, train], ignore_index=True)[var_list+[pa,'Y']]
    messydata=messydata.rename(columns={pa:'S'})
    messydata['S']=messydata['S'].replace(pa_dict)
    messydata['Y']=messydata['Y'].replace(label_dict)
    messydata=messydata[(messydata['S']==0)|(messydata['S']==1)]
    for col in var_list+['S','Y']:
        messydata[col]=messydata[col].astype('int64')
    messydata['W']=1
    bins_capitalgain=[100,3500,7500,10000]
    bins_capitalloss=[100,1600,1900,2200]
    bins_age=[26,36,46,56]
    bins_hours=[21,36,46,61]

    messydata=categerise(messydata,'age',bins_age)
    # messydata=categerise(messydata,'hours-per-week',bins_hours)
    messydata=categerise(messydata,'capital-gain',bins_capitalgain)
    messydata=categerise(messydata,'capital-loss',bins_capitalloss)
    
    return messydata

def categerise(df,col,bins):
    for i in range(len(bins)+1):
        if i == 0:
            df.loc[df[col] < bins[i], col] = i
        elif i == len(bins):
            df.loc[df[col] >= bins[i-1], col] = i
        else:
            df.loc[(df[col] >= bins[i-1])& (df[col] < bins[i]), col] = i        
    return df

def choose_x(var_list,messydata):
    tv_dist=dict()
    for x_name in var_list:
        x_range_single=list(pd.pivot_table(messydata,index=x_name,values=['W'])[('W')].index) 
        dist=rdata_analysis(messydata,x_range_single,x_name)
        tv_dist[x_name]=sum(abs(dist['x_0']-dist['x_1']))/2
    x_list=[]
    for key,val in tv_dist.items():
        if val>0.1:
            x_list+=[key]  
    return x_list,tv_dist

In [8]:
#TODO: change this path
data_path='C://personal//work//repair//.venv//Lib//site-packages//aif360//data//raw//adult'

var_list=['age','capital-gain','capital-loss','education-num'] #'hours-per-week',
pa='sex'
favorable_label = 1
var_dim=len(var_list)

K=200
e=0.01

if pa == 'sex':
    thresh=0.05
elif pa == 'race':
    thresh=0.1

messydata = load_data(data_path,var_list,pa)
x_list,tv_dist = choose_x(var_list,messydata)

X=messydata[var_list+['S','W']].to_numpy() # [X,S,W]
y=messydata['Y'].to_numpy() #[Y]

In [9]:
tv_dist

{'age': np.float64(0.1010227688866829),
 'capital-gain': np.float64(0.036924675713792855),
 'capital-loss': np.float64(0.020068855964263464),
 'education-num': np.float64(0.07095473385227195)}

In [10]:
thresh=0.05
x_list = ['age','education-num']
methods=['origin','barycentre','partial']
report=pd.DataFrame(columns=['DI','f1 macro','f1 micro','f1 weighted','TV distance','method'])
for ignore in range(10):
    # train val test 4:2:4
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4)
    X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.3)
    
    clf=RandomForestClassifier(max_depth=5).fit(X_train[:,0:var_dim],y_train)
    projpost = Projpostprocess(X_test,y_test,x_list,var_list,clf,K,e,thresh,favorable_label,linspace_range=(0.1,0.9),theta=1e-3)
    for method in methods[:-1]:
        report = pd.concat([report,projpost.postprocess(method)], ignore_index=True)
    
    for p in [1e-2,1e-3,1e-4]:
        report = pd.concat([report,projpost.postprocess('partial',para=p)], ignore_index=True)

report.to_csv(path+'/data/E3_postprocess_adult_'+str(pa)+'.csv',index=None)

In [11]:
report.dropna()

,DI,f1 macro,f1 micro,f1 weighted,TV distance,method
0,0.556828,0.674781,0.817935,0.787553,0.143438,origin
1,0.570718,0.674739,0.81512,0.78642,0.000413,barycentre
2,1.135559,0.59358,0.72959,0.716461,0.096578,partial_0.01
3,1.183006,0.571304,0.696217,0.692249,0.027948,partial_0.001
4,1.184253,0.567464,0.688949,0.687271,0.004101,partial_0.0001
5,0.604936,0.678033,0.816041,0.789866,0.122409,origin
6,0.575599,0.679785,0.814096,0.789809,0.001011,barycentre
7,1.107567,0.591966,0.725393,0.715755,0.08259,partial_0.01
8,1.170572,0.575154,0.694733,0.694733,0.02859,partial_0.001
9,1.091528,0.593651,0.691457,0.699416,0.003893,partial_0.0001


In [12]:
thresh=0.3
x_list = ['age','education-num']
methods=['origin','barycentre','partial'] # Place ROC in the end
report=pd.DataFrame(columns=['DI','f1 macro','f1 micro','f1 weighted','TV distance','method'])
for ignore in range(5):
    # train val test 4:2:4
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4)
    # X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.3)

    clf=RandomForestClassifier(max_depth=5).fit(X_train[:,0:var_dim],y_train)
    projpost = Projpostprocess(X_test,y_test,x_list,var_list,clf,K,e,thresh,favorable_label,linspace_range=(0.1,0.9),theta=1e-3)
    
    print(projpost.postprocess('origin'))
    print(projpost.postprocess('barycentre'))
    for t in range(2,4):
        print(projpost.postprocess('partial',para=10**(-t)))

# report.to_csv(path+'/data/E3_postprocess_adult_'+str(pa)+'.csv',index=None)

         DI  f1 macro  f1 micro f1 weighted TV distance  method
0  0.584437  0.680724  0.815171    0.789518    0.133415  origin
         DI  f1 macro  f1 micro f1 weighted TV distance      method
0  0.607646  0.676499  0.814148    0.787307    0.001001  barycentre
         DI  f1 macro  f1 micro f1 weighted TV distance        method
0  0.621936  0.663651  0.810155    0.780216    0.091612  partial_0.01
         DI  f1 macro  f1 micro f1 weighted TV distance         method
0  0.630075  0.662343  0.808415    0.778962    0.027117  partial_0.001
         DI f1 macro  f1 micro f1 weighted TV distance  method
0  0.560664  0.67923  0.816553     0.78931    0.125442  origin
         DI  f1 macro  f1 micro f1 weighted TV distance      method
0  0.602166  0.668661  0.813431    0.783533    0.000376  barycentre
         DI  f1 macro  f1 micro f1 weighted TV distance        method
0  0.596975  0.663186  0.811742    0.780508    0.085668  partial_0.01
         DI  f1 macro f1 micro f1 weighted TV distan

In [13]:
valpost = Projpostprocess(X_val,y_val,x_list,var_list,clf,K,e,'auto',linspace_range=(0.1,0.9),theta=1e-3)
valpost.thresh

Optional threshold =  [0.1        0.18888889 0.27777778 0.36666667 0.45555556 0.54444444
 0.63333333 0.72222222 0.81111111 0.9       ]
Disparate Impact =  [0.62509286 0.59580458 0.50576361 0.4952094  0.50101764 0.49892025
 0.52416662 0.52479874 0.52479874 0.52464492]
f1 scores =  [0.67938522 0.65948651 0.64807868 0.64907235 0.64133333 0.64080271
 0.62989884 0.62224281 0.62224281 0.61144746]


np.float64(0.1)

In [14]:
methods=['origin','unconstrained','barycentre','partial','ROC'] # Place ROC in the end
report=pd.DataFrame(columns=['DI','f1 macro','f1 micro','f1 weighted','TV distance','method'])
for ignore in range(10):
    # train val test 4:2:4
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4)
    X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.3)

    clf=RandomForestClassifier(max_depth=5).fit(X_train[:,0:var_dim],y_train)
    projpost = Projpostprocess(X_test,y_test,x_list,var_list,clf,K,e,thresh,favorable_label,linspace_range=(0.1,0.9),theta=1e-3)
    for method in methods[:-1]:
        # report = pd.concat([report,projpost.postprocess(method,para=1e-2)], ignore_index=True)
        report = pd.concat([report,projpost.postprocess(method,para=1e-3)], ignore_index=True)

    ROCpost = ROCpostprocess(X_val,y_val,var_list,clf,favorable_label) # use validation set to train a ROC model
    report = pd.concat([report,ROCpost.postprocess(X_test,y_test,tv_origin=projpost.tv_origin)], ignore_index=True)

report.to_csv(path+'/data/report_postprocess_adult_'+str(pa)+'.csv',index=None)

Optimal classification threshold (with fairness constraints) = 0.2900
Optimal ROC margin = 0.0322
Optimal classification threshold (with fairness constraints) = 0.1900
Optimal ROC margin = 0.0211
Optimal classification threshold (with fairness constraints) = 0.2900
Optimal ROC margin = 0.0322
Optimal classification threshold (with fairness constraints) = 0.2900
Optimal ROC margin = 0.0322
Optimal classification threshold (with fairness constraints) = 0.3100
Optimal ROC margin = 0.0344
Optimal classification threshold (with fairness constraints) = 0.3100
Optimal ROC margin = 0.0344
Optimal classification threshold (with fairness constraints) = 0.1900
Optimal ROC margin = 0.0211
Optimal classification threshold (with fairness constraints) = 0.2700
Optimal ROC margin = 0.0300
Optimal classification threshold (with fairness constraints) = 0.2900
Optimal ROC margin = 0.0322
Optimal classification threshold (with fairness constraints) = 0.1900
Optimal ROC margin = 0.0211


In [15]:
pa = 'race'
label_map = {1.0: '>50K', 0.0: '<=50K'}
privileged_groups = [{pa: 1}]
unprivileged_groups = [{pa: 0}]
if pa == 'sex':
    thresh=0.05
    protected_attribute_maps = [{1.0: 'Male', 0.0: 'Female'}]
    cd = AdultDataset(protected_attribute_names=[pa],privileged_classes=[['Male'],[1.0]], 
        metadata={'label_map': label_map,'protected_attribute_maps': protected_attribute_maps},
        # categorical_features=['workclass', 'marital-status', 'occupation', 'relationship', 'native-country']
        features_to_drop=['race','fnlwgt','education','relationship',
                          'native-country','workclass','marital-status','occupation'])
elif pa == 'race':
    thresh=0.1
    protected_attribute_maps = [{1.0: 'White', 0.0:'Non-white'}]
    cd = AdultDataset(protected_attribute_names=[pa],privileged_classes=[['White'],[1.0]],
        metadata={'label_map': label_map,'protected_attribute_maps': protected_attribute_maps}, #
        features_to_drop=['sex','fnlwgt','education','relationship',
                          'native-country','workclass','marital-status','occupation'])
    #,'workclass','marital-status','occupation','relationship',

# train,test = cd.split([0.6], shuffle=True) #len(test.instance_names) = 2057
var_list = cd.feature_names.copy()
var_list.remove(pa)
var_dim=len(var_list)

K=200
e=0.01
bins_capitalgain=[100,3500,7500,10000]
bins_capitalloss=[100,1600,1900,2200]

messydata=cd.convert_to_dataframe()[0]
messydata=messydata.rename(columns={pa:'S',cd.label_names[0]:'Y'})
messydata=messydata[(messydata['S']==1)|(messydata['S']==0)]
for col in var_list+['S','Y']:
    messydata[col]=messydata[col].astype('int64')
messydata['W']=cd.instance_weights
# project 0-100 to {0,1,...,5}
messydata['age']=np.floor((messydata['age'].to_numpy()-17)/15)
messydata['hours-per-week']=np.floor(messydata['hours-per-week'].to_numpy()/20)
messydata=categerise(messydata,'capital-gain',bins_capitalgain)
messydata=categerise(messydata,'capital-loss',bins_capitalloss)

X=messydata[var_list+['S','W']].to_numpy() # [X,S,W]
y=messydata['Y'].to_numpy() #[Y]
tv_dist=dict()
for x_name in var_list:
    x_range_single=list(pd.pivot_table(messydata,index=x_name,values=['W'])[('W')].index) 
    dist=rdata_analysis(messydata,x_range_single,x_name)
    tv_dist[x_name]=sum(abs(dist['x_0']-dist['x_1']))/2
x_list=[]
for key,val in tv_dist.items():
    if val>0.045:
        x_list+=[key]        
tv_dist

{'age': np.float64(0.04744857663969924),
 'education-num': np.float64(0.056762405582129784),
 'capital-gain': np.float64(0.021127950774052707),
 'capital-loss': np.float64(0.011363681253224836),
 'hours-per-week': np.float64(0.04445283428803036)}